In [ ]:
from v1 to v2 real function to implement in prepocess pipeline <br>
from v2 to v3 solve the header pb n computeStructural Image <br>
from v3 to v4 to find why the results are different <br>
v6 prendre en compte slope et intercept <br>
V7 passage en fslmaths pour voir <br>
<br>
array_header.get_slope_inter()

In [1]:
img1 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c1t0009_t1_s03.nii'
img2 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c2t0009_t1_s03.nii'
img3 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/mt0009_t1_s03.nii'


In [2]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

from nipype.interfaces.utility import Function, IdentityInterface

from nipype.interfaces.fsl.maths import MathsCommand
from nipype.interfaces.fsl.utils import PlotMotionParams   # to plot moco variable

from nipype import Node

In [37]:
computeStruct = pe.Workflow(name='computeStruct')

In [38]:
inputNode = Node(IdentityInterface(fields=['gm_file', 'wm_file', 't1_file']), name="inputNode")

In [39]:
# ii.inputs.a = 'foo'

inputNode.inputs.gm_file = img1
inputNode.inputs.wm_file = img2
inputNode.inputs.t1_file = img3


In [40]:
from nipype.interfaces.fsl import MultiImageMaths

addFiles = pe.Node(interface=MultiImageMaths(), name='addFiles')
    
addFiles.inputs.op_string = "-add %s"   
addFiles.inputs.output_datatype = 'short'
addFiles.inputs.ignore_exception = False     
addFiles.inputs.output_type = 'NIFTI'     
addFiles.inputs.terminal_output = 'stream'     

computeStruct.connect(inputNode,"gm_file" , addFiles, "in_file")
computeStruct.connect(inputNode,"wm_file" , addFiles, "operand_files")

In [41]:
from nipype.interfaces.fsl import Threshold

thrFile = pe.Node(interface=Threshold(), name='thrFile')
      
thrFile.inputs.thresh = 0.2   
thrFile.inputs.ignore_exception = False     
thrFile.inputs.output_type = 'NIFTI'     
thrFile.inputs.terminal_output = 'stream'     

computeStruct.connect(addFiles,"out_file" , thrFile, "in_file")

In [42]:
from nipype.interfaces.fsl import ApplyMask

maskFiles = pe.Node(interface=ApplyMask(), name='maskFiles')
  
maskFiles.inputs.ignore_exception = False     
maskFiles.inputs.output_type = 'NIFTI'     
maskFiles.inputs.terminal_output = 'stream' 
maskFiles.inputs.output_datatype ='short'

computeStruct.connect(inputNode,"t1_file" , maskFiles, "in_file")
computeStruct.connect(thrFile,"out_file" , maskFiles, "mask_file")


In [43]:
outputNode = Node(IdentityInterface(fields=['struct_file']), name="outputNode")

computeStruct.connect(maskFiles,  'out_file', outputNode, 'struct_file')

computeStruct.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node addFiles in dir: /tmp/tmpdTIeDS/computeStruct/addFiles
INFO:workflow:Running: fslmaths /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c1t0009_t1_s03.nii -add /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c2t0009_t1_s03.nii /tmp/tmpdTIeDS/computeStruct/addFiles/c1t0009_t1_s03_maths.nii -odt short
INFO:workflow:Executing node thrFile in dir: /tmp/tmpkksV2g/computeStruct/thrFile
INFO:workflow:Running: fslmaths /tmp/tmpdTIeDS/computeStruct/addFiles/c1t0009_t1_s03_maths.nii -thr 0.2000000000 /tmp/tmpkksV2g/computeStruct/thrFile/c1t0009_t1_s03_maths_thresh.nii
INFO:workflow:Executing node maskFiles in dir: /tmp/tmpQ58Eyi/computeStruct/maskFiles
INFO:workflow:Running: fslmaths /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/mt0009_t1_s03.nii -mas /tmp/tmpkksV2g/computeStruct/thrFile/c1t0009_t1_s03_maths_thresh.nii /tmp/tmpQ58Eyi/computeStruct/maskFiles/mt0009_t1

In [26]:
%cd /tmp/tmpjcmbhy/computeStruct/maskFiles
%ls

/tmp/tmpjcmbhy/computeStruct/maskFiles
_0xbafef1332a4778b08f61381a56b35214.json  _inputs.pklz  result_maskFiles.pklz
c1t0009_t1_s03_masked.nii                 _node.pklz
command.txt                               _report/


In [ ]:
!fslmaths  /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c1t0009_t1_s03.nii -add /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c2t0009_t1_s03.nii add.nii  -odt short
!fslmaths add.nii -thr 0.2 thr.nii
!fslmaths   /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/mt0009_t1_s03.nii -mas thr.nii  struct.nii -odt short

img = nb.load(filename)
data = img.get_data()
if data.max() < 2**16 and data.min() >= 0:
    nb.Nifti1Image(data.astype(np.int16), img.affine, img.header).to_filename(fileout)
else:
    raise ValueError('data range cannot be accommodated in 16-bit format')
    
np.asarray(a, dtype=np.float32)

In [29]:
%cd /homes_unix/hirsch/_goldstd/goldstarResults5

/homes_unix/hirsch/_goldstd/goldstarResults5


>>> from nipype.interfaces.fsl import MultiImageMaths
>>> maths = MultiImageMaths()
>>> maths.inputs.in_file = "functional.nii"
>>> maths.inputs.op_string = "-add %s -mul -1 -div %s"
>>> maths.inputs.operand_files = ["functional2.nii", "functional3.nii"]
>>> maths.inputs.out_file = "functional4.nii"
>>> maths.cmdline 
'fslmaths functional.nii -add functional2.nii -mul -1 -div functional3.nii functional4.nii'

In [44]:
%cd /homes_unix/hirsch/_goldstd/goldstarResults6
!fslhd /scratch/user/hirsch/datadir3/data_set_batch5/t0009/repos01/T1/c1c2T1.nii > /homes_unix/hirsch/_goldstd/goldstarResults6/imgca1.txt
!fslhd /tmp/tmpQ58Eyi/computeStruct/maskFiles/mt0009_t1_s03_masked.nii >  /homes_unix/hirsch/_goldstd/goldstarResults6/imgca2.txt
!diff imgca1.txt imgca2.txt
!fslmaths /scratch/user/hirsch/datadir3/data_set_batch5/t0009/repos01/T1/c1c2T1.nii -sub /tmp/tmpQ58Eyi/computeStruct/maskFiles/mt0009_t1_s03_masked.nii diff_calc.nii
!fslstats diff_calc.nii -r

/homes_unix/hirsch/_goldstd/goldstarResults6
1c1
< filename       /scratch/user/hirsch/datadir3/data_set_batch5/t0009/repos01/T1/c1c2T1.nii
---
> filename       /tmp/tmpQ58Eyi/computeStruct/maskFiles/mt0009_t1_s03_masked.nii
29c29
< scl_slope      0.044728
---
> scl_slope      1.000000
66c66
< descrip        spm - algebra
---
> descrip        FSL5.0
-0.287547 0.345815 


In [ ]:
!fslhd /scratch/user/hirsch/datadir3/data_set_batch5/t0009/repos01/T1/c1c2T1.nii > /homes_unix/hirsch/_goldstd/goldstarResults5/imgca1.txt
!fslhd struct.nii >  imgca2.txt
!diff imgca1.txt imgca2.txt

In [30]:
!fslmaths /scratch/user/hirsch/datadir3/data_set_batch5/t0009/repos01/T1/c1c2T1.nii -sub struct.nii diff_calc.nii
!fslstats diff_calc.nii -r

-0.287547 0.345815 


In [ ]:
!fslview diff_calc

In [ ]:
!fslhd /scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c1t0009_t1_s03.nii > /homes_unix/hirsch/_goldstd/goldstarResults4/img_gm1.txt

## essai transposition pour avoir les voxels de gm et voir les différences dans les 2 calculs

In [ ]:
import numpy as np     
import nibabel as nib     
import os
# on regarde le grey matter
img1 = '/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/c1t0009_t1_s03.nii'
i1=nib.load(img1)         
i1array=np.asarray(i1.dataobj).copy() 
i1array[(10 * i1array )< 2] = 0
    # binary mask the resulting image
i1array[i1array > 0] = 1
    

In [ ]:
np.transpose(np.nonzero(i1array))

In [ ]:
print r1array[24,  98, 114]
print r2array[24,  98, 114]

In [ ]:
print r1array[158, 111, 116]
print r2array[158, 111, 116]

In [ ]:
# x[np.where( x > 3.0 )] 
diff_res=r2array[np.where(( r1array-r2array) > 1.03 )]
print np.count_nonzero(diff_res)
# pas bon

In [ ]:
diff_res = r1array - r2array
print np.count_nonzero(diff_res)

In [ ]:
diff_res.shape

In [ ]:
180*256*256 - 1166481

In [ ]:
np.amax(diff_res) 

In [ ]:
print np.amin(r1array) 
print np.amax(r1array) 

In [ ]:
np.transpose(np.nonzero(diff_res))

In [ ]:
res1 ='/scratch/user/hirsch/goldstd/data_set/t0009/repos01/T1/dmt0009_t1_s02.nii'
res2= 'struct_image.nii'

In [ ]:
import numpy as np     
import nibabel as nib     
import os 
r1=nib.load(res1)         
r1array=np.asarray(r1.dataobj).copy() # Avoid caching the proxy image
    
r2=nib.load(res2)         
r2array=np.asarray(r2.dataobj).copy()

In [ ]:
np.array_equal(i1array, i2array)

In [ ]:
#np.allclose(r1array, r2array, rtol=1e-01)
np.allclose(r1array, r2array, rtol=1)

In [ ]:
%cd /homes_unix/hirsch/_pypipe/datadir/data_results/structural

In [ ]:
# test
img1 = 'c1t0009_t1_s03.nii'
img2 = 'c2t0009_t1_s03.nii'
img3 = 'mt0009_t1_s03.nii'
result = computeStructuralImage(img1,img2,img3)

In [ ]:
i1=nb.load('c1t0009_t1_s03.nii')         
i1array=np.asarray(i1.dataobj).copy() # Avoid caching the proxy image, thanks G. V. !!

In [ ]:
i2=nb.load('c1t0009_t1_s03.nii')         
i2array=np.asarray(i2.dataobj).copy() 

In [ ]:
i3=nb.load('BiasField_t0009_t1_s03.nii')         
i3array=np.asarray(i3.dataobj).copy() 

In [ ]:
gi = i1array + i2array

In [ ]:
gi[90,:, 128]

In [ ]:
# threshold image wm + csf
# a[a > 10] = 0
gi[gi < 0.2] = 0

In [ ]:
# mask the resulting image
gi[gi > 0.0] = 1

In [ ]:
gi[90,:, 128]

In [ ]:
# numpy.multiply(x1, x2[, out])
struct_image_array = np.multiply(gi, i3array)

In [ ]:
wm_csf = gi[gi > 0.2]

In [ ]:
180*256*256

In [ ]:
gi[90,128, 128]

In [ ]:
i3array[90,128, 128]

In [ ]:
i3array[90,0, 128]

In [ ]:
struct_image_array[90,128, 128]

In [ ]:
struct_image_array[90,0, 128]

In [ ]:
dims= i1array.shape
print dims

In [ ]:
i1.header.get_data_shape()

In [ ]:
i2=nib.load('c1t0009_t1_s03.nii')

In [ ]:
i3=nib.load('c2t0009_t1_s03.nii')

In [ ]:
data2 = i2.get_data()

In [ ]:
data2.__class__.__name__

In [ ]:
data3 = i3.get_data()

In [ ]:
data = data2 + data3

In [ ]:
data_threashold = data[data>0.2]

In [ ]:
data_threashold

In [ ]:
data_threashold[10]

In [ ]:
data_threashold.head()

In [ ]:
len(data_threashold)

In [ ]:
nib.load()?